In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import os
import seaborn as sns

import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics

import torch # Libreria
from torch.utils.data import Dataset # Clase u objeto que va a contener la informacion que vamos a utilizar para entrenar y evaluar nuestro algoritmo
from torch.utils.data import DataLoader

In [2]:
device = ""
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

print(torch.cuda.is_available())

True


In [3]:
if torch.cuda.is_available():
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 7605
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 2060
__CUDA Device Total Memory [GB]: 6.442450944


In [4]:
print("Device: ",device)

Device:  cuda:0


## Lectura del Dataset

In [56]:
PATH = r'./DLModel/ '

In [5]:
path = './World Happiness Report/'

In [6]:
data_csv = os.listdir(path)
print(data_csv)

['2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv', 'ds_final.csv', 'full_2015.csv', 'full_2016.csv', 'full_2017.csv', 'full_2018.csv', 'full_2019.csv', 'full_data.csv']


In [7]:
ds = pd.read_csv(path+'ds_final.csv')

In [8]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       782 non-null    float64
 1   Country                    782 non-null    object 
 2   Happiness Score            782 non-null    float64
 3   GDP per Capita             782 non-null    float64
 4   Life Expectancy            782 non-null    float64
 5   Freedom                    782 non-null    float64
 6   Perceptions of corruption  782 non-null    float64
 7   Generosity                 782 non-null    float64
 8   Region                     782 non-null    object 
dtypes: float64(7), object(2)
memory usage: 55.1+ KB


In [9]:
ds.head()

,Year,Country,Happiness Score,GDP per Capita,Life Expectancy,Freedom,Perceptions of corruption,Generosity,Region
0,2015.0,Switzerland,7.587,1.39651,0.94143,0.66557,0.41978,0.29678,Western Europe
1,2015.0,Iceland,7.561,1.30232,0.94784,0.62877,0.14145,0.43630,Western Europe
2,2015.0,Denmark,7.527,1.32548,0.87464,0.64938,0.48357,0.34139,Western Europe
3,2015.0,Norway,7.522,1.45900,0.88521,0.66973,0.36503,0.34699,Western Europe
4,2015.0,Canada,7.427,1.32629,0.90563,0.63297,0.32957,0.45811,North America


#### OHE sobre la feature Country y Region

In [10]:
country = pd.get_dummies(ds.Country, prefix=None)
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Columns: 165 entries, Afghanistan to Zimbabwe
dtypes: uint8(165)
memory usage: 126.1 KB


In [11]:
country.head()

,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahrain,...,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
region = pd.get_dummies(ds.Region, prefix=None)
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype
---  ------                           --------------  -----
 0   Australia and New Zealand        782 non-null    uint8
 1   Central and Eastern Europe       782 non-null    uint8
 2   Eastern Asia                     782 non-null    uint8
 3   Latin America and Caribbean      782 non-null    uint8
 4   Middle East and Northern Africa  782 non-null    uint8
 5   North America                    782 non-null    uint8
 6   Southeastern Asia                782 non-null    uint8
 7   Southern Asia                    782 non-null    uint8
 8   Sub-Saharan Africa               782 non-null    uint8
 9   Western Europe                   782 non-null    uint8
dtypes: uint8(10)
memory usage: 7.8 KB


In [13]:
region.head()

,Australia and New Zealand,Central and Eastern Europe,Eastern Asia,Latin America and Caribbean,Middle East and Northern Africa,North America,Southeastern Asia,Southern Asia,Sub-Saharan Africa,Western Europe
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0


In [14]:
ds_ohe = pd.concat([ds, country, region], axis=1)
ds_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Columns: 184 entries, Year to Western Europe
dtypes: float64(7), object(2), uint8(175)
memory usage: 188.8+ KB


In [15]:
ds_ohe.head()

,Year,Country,Happiness Score,GDP per Capita,Life Expectancy,Freedom,Perceptions of corruption,Generosity,Region,Afghanistan,...,Australia and New Zealand,Central and Eastern Europe,Eastern Asia,Latin America and Caribbean,Middle East and Northern Africa,North America,Southeastern Asia,Southern Asia,Sub-Saharan Africa,Western Europe
0,2015.0,Switzerland,7.587,1.39651,0.94143,0.66557,0.41978,0.29678,Western Europe,0,...,0,0,0,0,0,0,0,0,0,1
1,2015.0,Iceland,7.561,1.30232,0.94784,0.62877,0.14145,0.43630,Western Europe,0,...,0,0,0,0,0,0,0,0,0,1
2,2015.0,Denmark,7.527,1.32548,0.87464,0.64938,0.48357,0.34139,Western Europe,0,...,0,0,0,0,0,0,0,0,0,1
3,2015.0,Norway,7.522,1.45900,0.88521,0.66973,0.36503,0.34699,Western Europe,0,...,0,0,0,0,0,0,0,0,0,1
4,2015.0,Canada,7.427,1.32629,0.90563,0.63297,0.32957,0.45811,North America,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
final_data = ds_ohe.drop(['Year', 'Country', 'Region', 'Happiness Score'], axis=1, inplace=False)
output = ds_ohe['Happiness Score']

In [17]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Columns: 180 entries, GDP per Capita to Western Europe
dtypes: float64(5), uint8(175)
memory usage: 164.3 KB


In [18]:
final_data = final_data.to_numpy()
output = output.to_numpy()

In [19]:
normalized_data = np.float32(final_data - np.min(final_data, axis=0)) / (np.max(final_data, axis=0) - np.min(final_data, axis=0))

In [20]:
normalized_data.shape[1]

180

### Dataset Split

In [21]:
n = final_data.shape[0]
idx = np.random.permutation(np.arange(0, n))
train_idx = idx[:int(0.8 * n)]
test_idx = idx[int(0.8 * n):]
assert len(train_idx) + len(test_idx) == len(idx)

In [22]:
x_train = final_data[train_idx,:]
y_train = output[train_idx]

x_test = final_data[test_idx,:]
y_test = output[test_idx]

In [23]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(625, 180)
(625,)
(157, 180)
(157,)


### Deep Learning

In [24]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]
  
    def __getitem__(self, idx):
        return self.x[idx,:], self.y[idx]

In [25]:
training_set = CustomDataset(x_train, y_train)

In [26]:
test_set = CustomDataset(x_test, y_test)

In [27]:
training_dataloader = DataLoader(training_set, shuffle=True)
test_dataloader = DataLoader(test_set, shuffle=True)

In [28]:
# Definimos el modelo
class NNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = torch.nn.Linear(in_features = normalized_data.shape[1], out_features = 300, bias = True)
        self.dropout_1 = torch.nn.Dropout(p=0.05)
        self.relu_1 = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(in_features = 300, out_features = 500, bias = True)
        self.dropout_2 = torch.nn.Dropout(p=0.25)
        self.relu_2 = torch.nn.ReLU()
        self.linear_3 = torch.nn.Linear(in_features = 500, out_features = 800, bias = True)
        self.dropout_3 = torch.nn.Dropout(p=0.25)
        self.relu_3 = torch.nn.ReLU()
        self.linear_4 = torch.nn.Linear(in_features = 800, out_features = 200, bias = True)
        self.dropout_4 = torch.nn.Dropout(p=0.05)
        self.relu_4 = torch.nn.ReLU()
        self.linear_5 = torch.nn.Linear(in_features = 200, out_features = 1, bias = True)
        
    def forward(self, x):
                                    
        z1 = self.linear_1(x)
        z1_1 = self.dropout_1(z1)
        a1 = self.relu_1(z1_1)
        z2 = self.linear_2(a1)
        z2_1 = self.dropout_2(z2)
        a2 = self.relu_2(z2_1)
        z3 = self.linear_3(a2)
        z3_1 = self.dropout_3(z3)
        a3 = self.relu_3(z3_1)
        z4 = self.linear_4(a3)
        z4_1 = self.dropout_4(z4)
        a4 = self.relu_4(z4_1)
        y = self.linear_5(a4)

        return y

In [29]:
nnet = NNet()

In [31]:
nnet.to(device)

NNet(
  (linear_1): Linear(in_features=180, out_features=300, bias=True)
  (dropout_1): Dropout(p=0.05, inplace=False)
  (relu_1): ReLU()
  (linear_2): Linear(in_features=300, out_features=500, bias=True)
  (dropout_2): Dropout(p=0.25, inplace=False)
  (relu_2): ReLU()
  (linear_3): Linear(in_features=500, out_features=800, bias=True)
  (dropout_3): Dropout(p=0.25, inplace=False)
  (relu_3): ReLU()
  (linear_4): Linear(in_features=800, out_features=200, bias=True)
  (dropout_4): Dropout(p=0.05, inplace=False)
  (relu_4): ReLU()
  (linear_5): Linear(in_features=200, out_features=1, bias=True)
)

In [36]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(nnet.parameters(), lr=0.000001)

In [37]:
for epoch in range(500):
    running_loss = 0
    for i, data in enumerate(training_dataloader):
        # data
        x, y = data
        x = x.to(device).float()
        y = y.to(device).float().reshape(-1,1)
        
        # set gradient to zero
        optimizer.zero_grad()

        #forward
        y_hat = nnet(x).reshape(-1,1)

        #loss
        loss = criterion(y_hat, y)

        #backward
        loss.backward()

        #update of parameters
        optimizer.step()

        #compute metrics and statistics
        running_loss += loss.item()
    
    nnet.eval()
    with torch.no_grad():
        nnet_test_scores = []
        truth = []
        
        for i, data in enumerate(test_dataloader):
            #batch
            x, y = data
            x = x.to(device).float()
            y = y.to(device).float().reshape(-1,1)
        
            # forward 
            y_hat = nnet(x)
            #y_hat = torch.abs(y_hat)
        
            # accumulate data
            truth = list(y.cpu().detach().numpy()) 
            nnet_test_scores = list(y_hat.cpu().detach().numpy())
            
        mse = metrics.mean_squared_error(truth, nnet_test_scores)
        
    print(f"Epoch = {epoch} - loss = {running_loss / len(training_set)} - mse: {mse}")

Epoch = 0 - loss = 0.03458808021203695 - mse: 0.0012544371420517564
Epoch = 1 - loss = 0.034656085224390336 - mse: 0.00031295939697884023
Epoch = 2 - loss = 0.03472687020277126 - mse: 0.17416654527187347
Epoch = 3 - loss = 0.03473829755797051 - mse: 0.00018509922665543854
Epoch = 4 - loss = 0.034750467008272244 - mse: 0.0007068690611049533
Epoch = 5 - loss = 0.034702992430118776 - mse: 0.004536781460046768
Epoch = 6 - loss = 0.03467613376344416 - mse: 6.904553447384387e-05
Epoch = 7 - loss = 0.03455494527449227 - mse: 0.02213672734797001
Epoch = 8 - loss = 0.034545272374002525 - mse: 0.006935890298336744
Epoch = 9 - loss = 0.03471467883454825 - mse: 1.3096723705530167e-08
Epoch = 10 - loss = 0.03462943159889837 - mse: 0.04895876348018646
Epoch = 11 - loss = 0.03459202286119034 - mse: 0.0013100291835144162
Epoch = 12 - loss = 0.03473437170179786 - mse: 6.741180186509155e-06
Epoch = 13 - loss = 0.03465460316153694 - mse: 0.019718971103429794
Epoch = 14 - loss = 0.034597480186372195 - mse

Epoch = 121 - loss = 0.03360026652519518 - mse: 0.13984645903110504
Epoch = 122 - loss = 0.033752420143250125 - mse: 0.0036887675523757935
Epoch = 123 - loss = 0.03379706678207658 - mse: 9.094947017729282e-11
Epoch = 124 - loss = 0.033729443371655086 - mse: 0.015573781915009022
Epoch = 125 - loss = 0.03378279503942904 - mse: 0.002249065786600113
Epoch = 126 - loss = 0.03376503199925283 - mse: 0.0025055056903511286
Epoch = 127 - loss = 0.033780847516228826 - mse: 0.0003000703873112798
Epoch = 128 - loss = 0.03379677201341256 - mse: 0.3334788382053375
Epoch = 129 - loss = 0.03370339391254493 - mse: 0.027691643685102463
Epoch = 130 - loss = 0.03364472569925201 - mse: 0.010791237466037273
Epoch = 131 - loss = 0.03362764423671069 - mse: 0.1642465591430664
Epoch = 132 - loss = 0.03368094653604712 - mse: 0.07622050493955612
Epoch = 133 - loss = 0.03370759625368701 - mse: 0.0038469911087304354
Epoch = 134 - loss = 0.03361759748929289 - mse: 0.1533525288105011
Epoch = 135 - loss = 0.03365269230

Epoch = 240 - loss = 0.03291107475641256 - mse: 0.02940071001648903
Epoch = 241 - loss = 0.03298815312720399 - mse: 0.3289636969566345
Epoch = 242 - loss = 0.03288663724309081 - mse: 0.008902615867555141
Epoch = 243 - loss = 0.0328596261402472 - mse: 0.002800966380164027
Epoch = 244 - loss = 0.03287266122532092 - mse: 0.5591953992843628
Epoch = 245 - loss = 0.03286818700144322 - mse: 0.04661443457007408
Epoch = 246 - loss = 0.03297396888448966 - mse: 0.19782298803329468
Epoch = 247 - loss = 0.03279788475524547 - mse: 0.5283380150794983
Epoch = 248 - loss = 0.032845677288606386 - mse: 0.006863644812256098
Epoch = 249 - loss = 0.03288072508908554 - mse: 0.014571339823305607
Epoch = 250 - loss = 0.03295742724234151 - mse: 0.018235620111227036
Epoch = 251 - loss = 0.032795718121790926 - mse: 0.004675090312957764
Epoch = 252 - loss = 0.032736098291780584 - mse: 0.027758019044995308
Epoch = 253 - loss = 0.03297203989195013 - mse: 0.09106497466564178
Epoch = 254 - loss = 0.03273080163781706 -

Epoch = 359 - loss = 0.0320986148105736 - mse: 0.20929336547851562
Epoch = 360 - loss = 0.0319784470800063 - mse: 0.019212568178772926
Epoch = 361 - loss = 0.03210190201581572 - mse: 0.006416554097086191
Epoch = 362 - loss = 0.03218847540161914 - mse: 0.1862175166606903
Epoch = 363 - loss = 0.032149643193305384 - mse: 0.005379446316510439
Epoch = 364 - loss = 0.03215877282832407 - mse: 0.05351044237613678
Epoch = 365 - loss = 0.032157780417212053 - mse: 0.005815965123474598
Epoch = 366 - loss = 0.03217903351698224 - mse: 0.026421718299388885
Epoch = 367 - loss = 0.0321313733095576 - mse: 0.21602961421012878
Epoch = 368 - loss = 0.03213086878020331 - mse: 0.02934236079454422
Epoch = 369 - loss = 0.032166692843882995 - mse: 0.14854294061660767
Epoch = 370 - loss = 0.03208826749852924 - mse: 0.018241673707962036
Epoch = 371 - loss = 0.032057631755181955 - mse: 0.05722878500819206
Epoch = 372 - loss = 0.03206541065054844 - mse: 0.14259198307991028
Epoch = 373 - loss = 0.03210456355828137 -

Epoch = 479 - loss = 0.03141281167478701 - mse: 0.0014782376820221543
Epoch = 480 - loss = 0.03141194858300441 - mse: 0.004499281290918589
Epoch = 481 - loss = 0.03142184251291365 - mse: 0.12326281517744064
Epoch = 482 - loss = 0.03140711503726088 - mse: 0.02746453694999218
Epoch = 483 - loss = 0.0314453070309105 - mse: 0.0013413489796221256
Epoch = 484 - loss = 0.031365807389472686 - mse: 0.007460104767233133
Epoch = 485 - loss = 0.03141901576165219 - mse: 0.059265196323394775
Epoch = 486 - loss = 0.03140581130495302 - mse: 0.025268996134400368
Epoch = 487 - loss = 0.03136803817701548 - mse: 0.014820014126598835
Epoch = 488 - loss = 0.03143374463884138 - mse: 0.13929635286331177
Epoch = 489 - loss = 0.031382019037674035 - mse: 0.07918446511030197
Epoch = 490 - loss = 0.03127825028236621 - mse: 0.21291661262512207
Epoch = 491 - loss = 0.031445298692211965 - mse: 0.01552276685833931
Epoch = 492 - loss = 0.031437397317769136 - mse: 0.032370589673519135
Epoch = 493 - loss = 0.031284792220

In [38]:
for i, data in enumerate(training_dataloader):
            #batch
            x, y = data
            x = x.to(device).float()
            y = y.to(device).float().reshape(-1,1)
            
            #print(x)
            # forward 
            y_hat = nnet(x)
            print(y_hat, y)

tensor([[4.0385]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[3.6670]], device='cuda:0')
tensor([[5.2695]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.1950]], device='cuda:0')
tensor([[5.3829]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.5240]], device='cuda:0')
tensor([[4.0972]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.0320]], device='cuda:0')
tensor([[7.2727]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.2720]], device='cuda:0')
tensor([[4.5558]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.5350]], device='cuda:0')
tensor([[4.1099]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.1890]], device='cuda:0')
tensor([[5.2295]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.3360]], device='cuda:0')
tensor([[7.5272]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.4950]], device='cuda:0')
tensor([[5.2056]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.1920]], device='cuda:0')
tensor([[5.7268]], device='cud

tensor([[7.5814]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.5040]], device='cuda:0')
tensor([[3.8080]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.0280]], device='cuda:0')
tensor([[3.9824]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.1610]], device='cuda:0')
tensor([[6.4326]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.3790]], device='cuda:0')
tensor([[6.0070]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.1230]], device='cuda:0')
tensor([[5.0725]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.1510]], device='cuda:0')
tensor([[5.9212]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.7910]], device='cuda:0')
tensor([[6.1445]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.2230]], device='cuda:0')
tensor([[6.5374]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.4760]], device='cuda:0')
tensor([[6.6818]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.8250]], device='cuda:0')
tensor([[4.4671]], device='cud

tensor([[3.9871]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[3.7630]], device='cuda:0')
tensor([[4.8602]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.7960]], device='cuda:0')
tensor([[6.5397]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.7860]], device='cuda:0')
tensor([[2.9769]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[3.0830]], device='cuda:0')
tensor([[5.1750]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.0740]], device='cuda:0')
tensor([[7.4293]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.3770]], device='cuda:0')
tensor([[5.9779]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.9200]], device='cuda:0')
tensor([[3.9310]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[3.9640]], device='cuda:0')
tensor([[5.6389]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.4770]], device='cuda:0')
tensor([[4.5734]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.5750]], device='cuda:0')
tensor([[7.1202]], device='cud

tensor([[6.4960]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.4850]], device='cuda:0')
tensor([[5.3732]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.4010]], device='cuda:0')
tensor([[5.8745]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.7790]], device='cuda:0')
tensor([[5.1705]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.8390]], device='cuda:0')
tensor([[7.2041]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.2460]], device='cuda:0')
tensor([[5.2417]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.1310]], device='cuda:0')
tensor([[5.2026]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.2650]], device='cuda:0')
tensor([[6.0153]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.0080]], device='cuda:0')
tensor([[4.4153]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.0150]], device='cuda:0')
tensor([[4.3746]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.2800]], device='cuda:0')
tensor([[6.6523]], device='cud

tensor([[7.3747]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.3780]], device='cuda:0')
tensor([[4.2038]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.0960]], device='cuda:0')
tensor([[6.4413]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.4220]], device='cuda:0')
tensor([[7.0490]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[7.0390]], device='cuda:0')
tensor([[6.0375]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.1410]], device='cuda:0')
tensor([[4.3282]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.7580]], device='cuda:0')
tensor([[6.4205]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[6.4030]], device='cuda:0')
tensor([[3.5360]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[3.2310]], device='cuda:0')
tensor([[4.3577]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[4.4190]], device='cuda:0')
tensor([[6.0273]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[5.9640]], device='cuda:0')
tensor([[4.6671]], device='cud

In [57]:
torch.save(nnet, PATH+'DL_Model.pth')